___

<a href='http://www.pieriandata.com'> <img src='../Pierian_Data_Logo.png' /></a>
___

# Tensorflow with ContribLearn

As we saw previously how to build a full Multi-Layer Perceptron model with full Sessions in Tensorflow. Unfortunately this was an extremely involved process. However developers have created ContribLearn (previously known as TKFlow or SciKit-Flow) which provides a SciKit Learn like interface for Tensorflow!

It is much easier to use, but you sacrifice some level of customization of your model. Let's go ahead and explore it!

## Get the Data

We will the iris data set.

Let's get the data:

In [1]:
from sklearn.datasets import load_iris

In [2]:
iris = load_iris()

In [22]:
X = iris['data']
X.shape


(150, 4)

In [5]:
y = iris['target']

In [10]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [11]:
set(y)

{0, 1, 2}

In [12]:
y.dtype

dtype('int64')

## Train Test Split

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [23]:
import numpy as np
X_train = np.array( X_train, dtype=np.float32 ) 
X_test = np.array( X_test, dtype=np.float32 )

# Contrib.learn

Let's show you how to use the simpler contrib.learn interface!

In [17]:
import tensorflow.contrib.learn as learn

In [24]:
feature_columns = learn.infer_real_valued_columns_from_input(X_train)

There are several high level abstraction calls to models in learn, you can explore them with Tab, but we will use DNNClassifier, which stands for Deep Neural Network:

In [29]:
classifier = learn.DNNClassifier(hidden_units=[10, 20, 10], n_classes=3,feature_columns=feature_columns)#,feature_columns=feature_columns)
#classifier.fit(X_train, y_train, steps=200, batch_size=32)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x103b55e10>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': None}


In [38]:

import tensorflow as tf 
def get_train_inputs():
    x = tf.constant(X_train)
    y = tf.constant(y_train)

    return x, y

classifier.fit(input_fn=get_train_inputs, steps=200)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /var/folders/wc/97m43k5n4knd4vkdthrfb27r0000gn/T/tmp93e__a8h/model.ckpt-200
INFO:tensorflow:Saving checkpoints for 201 into /var/folders/wc/97m43k5n4knd4vkdthrfb27r0000gn/T/tmp93e__a8h/model.ckpt.
INFO:tensorflow:loss = 0.0678489, step = 201
INFO:tensorflow:global_step/sec: 1008.16
INFO:tensorflow:loss = 0.0522544, step = 301 (0.100 sec)
INFO:tensorflow:Saving checkpoints for 400 into /var/folders/wc/97m43k5n4knd4vkdthrfb27r0000gn/T/tmp93e__a8h/model.ckpt.
INFO:tensorflow:Loss for final step: 0.0438777.


DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x103b55b00>, 'hidden_units': [10, 20, 10], 'feature_columns': (_RealValuedColumn(column_name='', dimension=4, default_value=None, dtype=tf.float32, normalizer=None),), 'optimizer': None, 'activation_fn': <function relu at 0x111fbe488>, 'dropout': None, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

In [34]:
#iris_predictions = classifier.predict(X_test)
def get_test_inputs():
    x = tf.constant(X_test)
    return x 

iris_predictions = list(classifier.predict_classes (input_fn=get_test_inputs))

INFO:tensorflow:Restoring parameters from /var/folders/wc/97m43k5n4knd4vkdthrfb27r0000gn/T/tmp93e__a8h/model.ckpt-200


In [35]:
from sklearn.metrics import classification_report,confusion_matrix

In [36]:
print(classification_report(y_test,iris_predictions))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        12
          1       1.00      0.89      0.94        19
          2       0.88      1.00      0.93        14

avg / total       0.96      0.96      0.96        45



# Great Job!